In [1]:
#CARGAMOS LIBRERIAS
import pandas as pd
import numpy as np
import ast

In [2]:
# Leer el contenido del archivo: user_reviews.json
user_reviews = []
with open('./australian_user_reviews.json', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        user_reviews.append(ast.literal_eval(line))

# Crear el DataFrame anidado
df_reviews_anidado = pd.DataFrame(user_reviews)

In [3]:
# Normalizar la columna 'reviews' 
df_reviews_anidado = df_reviews_anidado.explode('reviews')

In [4]:
# reseteo el index
df_reviews_anidado.reset_index(drop=True, inplace=True)

#eliminamos los vacios de la columna review
df_reviews_anidado = df_reviews_anidado.dropna(subset=['reviews'])


In [5]:

anidado= df_reviews_anidado['reviews']

#normalizar la columna 'reviews' desanidando
df_reviews= pd.json_normalize(anidado)

#quitamos las columnas que no voy a usar
df_reviews=df_reviews.drop(['last_edited','funny','helpful'], axis=1)
#quitamos las columnas que no voy a usar
df_reviews_anidado=df_reviews_anidado.drop(['user_url','reviews'], axis=1)



In [6]:
df_reviews_anidado.head()

,user_id
0,76561197970982479
1,76561197970982479
2,76561197970982479
3,js41637
4,js41637


In [7]:
df_reviews.head(4)

,posted,item_id,recommend,review
0,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,"Posted July 15, 2011.",22200,True,Its unique and worth a playthrough.
2,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...


In [8]:
# Concatena los DataFrames utilizando el índice
df_reviews_completo = pd.concat([df_reviews_anidado, df_reviews], axis=1)

In [9]:
df_reviews_completo.head()

,user_id,posted,item_id,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,True,Its unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...


In [10]:
# quito de posted "posted" para dejar solo los años
df_reviews_completo['posted']=df_reviews_completo['posted'].str.replace('Posted','').str.replace('.','')

C:\Users\lgtqa\AppData\Local\Temp\ipykernel_13076\4054448446.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_reviews_completo['posted']=df_reviews_completo['posted'].str.replace('Posted','').str.replace('.','')


In [11]:
df_reviews_completo.head()

,user_id,posted,item_id,recommend,review
0,76561197970982479,"November 5, 2011",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"July 15, 2011",22200,True,Its unique and worth a playthrough.
2,76561197970982479,"April 21, 2011",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"June 24, 2014",251610,True,I know what you think when you see this title ...
4,js41637,"September 8, 2013",227300,True,For a simple (it's actually not all that simpl...


In [49]:
#convertir a fecha los valores de posted por año dejando como NaN los errores
df_reviews_completo['posted'] = pd.to_datetime(df_reviews_completo['posted'], errors='coerce').dt.year


In [50]:
# Filtrar el DataFrame original para mantener solo las filas con no nulos
df_reviews_completo = df_reviews_completo.dropna(subset=['posted', 'user_id','item_id'])

In [52]:
df_reviews_completo.head()

,user_id,posted,item_id,recommend,review
0,76561197970982479,2011.0,1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,2011.0,22200,True,Its unique and worth a playthrough.
2,76561197970982479,2011.0,43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,2014.0,251610,True,I know what you think when you see this title ...
4,js41637,2013.0,227300,True,For a simple (it's actually not all that simpl...


In [53]:
# Guardamos df como csv
nombre_archivo = "./reviews.csv"

df_reviews_completo.to_csv(nombre_archivo, index=False)

In [54]:
#traigo el archivo para seguir trabajando
df_reviews=pd.read_csv('./reviews.csv')


In [55]:
df_reviews.head(4)

,user_id,posted,item_id,recommend,review
0,76561197970982479,2011.0,1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,2011.0,22200,True,Its unique and worth a playthrough.
2,76561197970982479,2011.0,43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,2014.0,251610,True,I know what you think when you see this title ...


# empiezo con la parte de sentiment analisis



In [56]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [57]:
# Descargar el recurso necesario para VADER
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\lgtqa\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [58]:
#lo inicializo 
analyzer = SentimentIntensityAnalyzer()

In [59]:
def analyze_sentiment(review):
    if pd.isna(review):
        return 1  # Valor 1 para reseñas ausentes
    sentiment = analyzer.polarity_scores(review)
    if sentiment['compound'] >= 0.05:
        return 2  # Valor 2 para reseñas positivas
    elif sentiment['compound'] <= -0.05:
        return 0  # Valor 0 para reseñas negativas
    else:
        return 1  # Valor 1 para reseñas neutras

# Aplicar la función analyze_sentiment a la columna review
df_reviews['sentiment_analysis'] = df_reviews['review'].apply(analyze_sentiment)


In [60]:
# Elimino la columna review
df_reviews=df_reviews.drop(columns=['review'], axis=1)

In [62]:
df_reviews.head(4)

,user_id,posted,item_id,recommend,sentiment_analysis
0,76561197970982479,2011.0,1250,True,2
1,76561197970982479,2011.0,22200,True,2
2,76561197970982479,2011.0,43110,True,2
3,js41637,2014.0,251610,True,2


In [37]:
# Guardo el nuevo df como csv
nombre_archivo = "../FastAPI/reviews_sentiment_analysis.csv"

df_reviews.to_csv(nombre_archivo, index=False)